# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# import
from bs4 import BeautifulSoup
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display


In [2]:
import pandas as pd

In [ ]:
%run helper.ipynb


In [3]:
# Initialization
# Initialization

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCL


In [7]:
choice =input('models: 1-openai,  2-llama, 3-gemini, 4-cla1ude Choose 1/2/3/4/')

models: 1-openai,  2-llama, 3-gemini, 4-cla1ude Choose 1/2/3/4/ 3


In [8]:
if choice =='1':
     MODEL= "gpt-4o-mini"
     openai = OpenAI()
elif choice =='2':
    MODEL="llama3.2"
    openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
elif choice=='3':
    MODEL='gemini-2.0-flash-exp'
    
    openai=OpenAI(
        api_key=google_api_key, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
elif choice=='4':
    MODEL="claude-3-haiku-20240307",
    openai= OpenAI(
        api_key=anthropic_api_key,  # Your Anthropic API keyi
        base_url="https://api.anthropic.com/v1/" ) # Anthropic's API endpoint)
elif choice =='5':
    # MODEL='deepseek-r1'
    # openai=OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    pass
else:
    pass
    # MODEL="phi4"
    # openai=OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
print('Your choice is: '+ MODEL + ' (if model is claude you must indicate its real name inside the calls)')

Your choice is: gemini-2.0-flash-exp (if model is claude you must indicate its real name inside the calls)


In [ ]:
database=sharedLink('database')

In [ ]:
geek=sharedLink('geek')

In [68]:
sysMessage='You are an assistant of the firm : "wargamer reseller and publisher co." and you are experts in tabletop wargame printed since year 1960'
sysMessage+='''You must look to our shelves products using the tool "webSearch" trying to satisfy customer requests . Provide to translate the dialogues as requested by customer.
Example of customer requests:
"I need the game ..." plus the exact  or aproximate name of the game asked for.
"I'm looking for a game about Napoleonic wars" in this case many games containing the world Napoleon in their title are eligible to satisfy the  request.
"I want a game from..." and the exact or approximate name of a publisher is given, in this case in this case many games containing publisher's name are eligible to satisfy the  request.
'''
sysMessage+='''The database in which you should research the requests is a pandas dataframe  structured with each row containing data about one game:
'id', 'name', 'yearpublished', 'publisher'
'''
# sysMessage+=f'''
# If the customer has a specific request that we cannot fulfill with products in our shelves, tell to the customer we'll try to find the product on the web and
# then search this site: {geek} to look for it, using one of the tools you have at your disposal to do it. If the product was found, communicate to customer that news and ask if he/she could wait some days to receive the product and 
# the maximum amount the customer would like to spend for it'''

In [30]:
sysMessage

'You are an assistant of the firm : "wargamer reseller and publisher co." and you are experts in tabletop wargame printed since year 1960You must look to our shelves products using the tool "webSearch" trying to satisfy customer requests , if our customer need an answer in a language different from English then translate it into the requested language before submitting the answer to him using the "translator" tool\nExample of customer requests:\n"I need the game ..." plus the exact  or aproximate name of the game asked for.\n"I\'m looking for a game about Napoleonic wars" in this case many games containing the world Napoleon in their title are eligible to satisfy the  request.\n"I want a game from..." and the exact or approximate name of a publisher is given, in this case in this case many games containing publisher\'s name are eligible to satisfy the  request.\nThe database in which you should research the requests is a pandas dataframe  structured with each row containing data about 

<b>Audio generation</b>

In [13]:
def talker(message):
    openaiLocal=OpenAI()
    response = openaiLocal.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

<b>Tools</b>

In [14]:
def getItem(game):
    game = game.upper()
    df= pd.read_csv('boardgames_ranks.csv')
    df['name']=df['name'].str.upper()
    
    df.set_index('name', inplace=True)
    try:
        gameId=int(df.at[game,'id'])
    except:
        gameId=0
    
    return gameId
   

In [15]:

webSearch = {
    "name": "getItem",
    "description": "Search or shelves and then the market to check wether the product is available. Call this whenever you need to know if an item is present in our database  , for example when a customer asks 'I want the game X' ",
    "parameters": {
        "type": "object",
        "properties": {
            "game": {
                "type": "string",
                "description": "The game the customer asked for",
            },
        },
        "required": ["game"],
        "additionalProperties": False
    }
}

In [52]:
translator = {
    "name": "translate",
    "description": "Translate a text in a user chosen language ",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The text to be translated",
            },
            'language':{
                'type':'string',
                'description':'language of translation'
            }
        },
        "required": ["text",'language'],
        "additionalProperties": False
    }
}

In [73]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name=='getItem':
        arguments = json.loads(tool_call.function.arguments)
        game = arguments.get('game')
        gameId = getItem(game)
        response = {
            "role": "tool",
            "content": json.dumps({'game' : game,"geekId": gameId}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response,game
    # elif tool_call.function.name=='translate':
    #     arguments = json.loads(tool_call.function.arguments)
    #     text = arguments.get('text')
    #     language=arguments.get('language')
        
    #     response = {
    #         "role": "tool",
    #         "content": json.dumps({'text' :text,'language':language}),
    #         "tool_call_id": message.tool_calls[0].id
    #     }
    #     return response,None
    

In [75]:
tools=[{"type": "function", "function": webSearch}]

<b>Chat function</b>

In [69]:
def chat_variant(history):
    
    messages = [{"role": "system", "content": sysMessage} ] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    #image = None
    reply=response.choices[0].message.content
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, game = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        
        response = openai.chat.completions.create(model=MODEL, messages=messages)
       
        
    reply = response.choices[0].message.content
    print(reply)
    
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history

In [76]:
def chat(message, history):
    messages = [{"role": "system", "content": sysMessage}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, game = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    talker(response.choices[0].message.content)
    return response.choices[0].message.content

In [77]:
 gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


In [70]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        with gr.Column():
            clear = gr.Button("Clear")
        #with gr.Column():
            #language= gr.Dropdown(["Italian", "Spanish","English"], label="Select Language", value="Italian")
    def do_entry(message, history):
        
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat_variant, inputs=[chatbot], outputs=[chatbot]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=False)

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Hi there! How can I help you today? Are you looking for a specific wargame, or do you have something else in mind?



Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2103, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1650, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get